iPSC: P9_01 and MN: M6_03 are samples that were sequenced to over 100 million reads each. I need to downsample them, in intervals of 5 million reads, maybe 5-10 iterations each. I'll use [`seqtk`](https://github.com/lh3/seqtk) for the downsampling.

After I downsample, then I'll need to create a manifest file and submit these data for the analysis pipeline - probably minus everything after mapping.

In [2]:
cd /home/obotvinnik/projects/singlecell_pnms/data

/oasis/tscc/scratch/obotvinnik/projects/singlecell_pnms/data


In [6]:
# !tail downsample*err*

In [7]:
! seqtk sample


Usage:   seqtk sample [-2] [-s seed=11] <in.fa> <frac>|<number>

Options: -s INT       RNG seed [11]
         -2           2-pass mode: twice as slow but with much reduced memory



In [8]:
# seed = 1

# commands = []

# old_folder = '/home/obotvinnik/projects/singlecell_pnms/data'
# new_folder = '/home/obotvinnik/projects/downsample_singlecell/data'

# samples = 'P9_01', 'M6_03'
# read_pair = 'R1', 'R2'

# for millions_of_reads in [1] + range(5, 105):
#     for i in range(10):
#         for sample in samples:
#             for read in read_pair:
#                 sampled_reads = int(millions_of_reads * 1e6)
#                 kwargs = dict(seed=seed, old_folder=old_folder, read=read, new_folder=new_folder, 
#                               sample=sample, sampled_reads=sampled_reads, i=i, millions_of_reads=millions_of_reads)
#                 command = 'time seqtk sample -s {seed} {old_folder}/{sample}_{read}.fastq.gz {sampled_reads} '\
#                     '| gzip - > {new_folder}/{sample}_{sampled_reads}reads_iteration{i}_{read}.fastq.gz'.format(**kwargs)
#                 commands.append(command)
#             seed += 1

Not enough memory allocated to 20M+ reads downsampling so need to increase the number of nodes

Re-do all data that only has size of 20 bytes (not done)

In [14]:
import os
import qtools


seed = 1

commands = []

old_folder = '/home/obotvinnik/projects/singlecell_pnms/data'
new_folder = '/home/obotvinnik/projects/downsample_singlecell/data'

samples = 'P9_01', 'M6_03'


read_pair = 'R1', 'R2'



for millions_of_reads in [1] + range(5, 105, 5):
    for i in range(10):
        for sample in samples:
            for read in read_pair:
#                 if millions_of_reads < 10:
#                     continue
                sampled_reads = int(millions_of_reads * 1e6)
                kwargs = dict(seed=seed, old_folder=old_folder, read=read, new_folder=new_folder, 
                              sample=sample, sampled_reads=sampled_reads, i=i, millions_of_reads=millions_of_reads)
                new_file = '{new_folder}/{sample}_{sampled_reads}reads_iteration{i}_{read}.fastq.gz'.format(**kwargs)
                if os.stat(new_file).st_size > 20:
                    continue
            
                command = 'time seqtk sample -s {seed} {old_folder}/{sample}_{read}.fastq.gz {sampled_reads} '\
                    '| gzip - > {new_file}'.format(new_file=new_file, **kwargs)
                commands.append(command)
            seed += 1
    if millions_of_reads == 45:
        sub = qtools.Submitter(commands, 'downsample45', array=True, queue='home-scrm', walltime='00:15:00', ppn=1)
        commands = []
    elif millions_of_reads == 60:
        sub = qtools.Submitter(commands, 'downsample60', array=True, queue='home-scrm', walltime='00:30:00', ppn=2)
        commands = []
    elif millions_of_reads == 75:
        sub = qtools.Submitter(commands, 'downsample75', array=True, queue='home-scrm', walltime='01:00:00', ppn=3)
        commands = []
    elif millions_of_reads == 100:
        sub = qtools.Submitter(commands, 'downsample100', array=True, queue='home-scrm', walltime='02:00:00', ppn=4)
        commands = []
        

running 95 tasks as an array-job.
Wrote commands to downsample45.sh
.Submitted script to queue home-scrm. Job ID: 4554888
running 94 tasks as an array-job.
Wrote commands to downsample60.sh
.Submitted script to queue home-scrm. Job ID: 4554889
running 118 tasks as an array-job.
Wrote commands to downsample75.sh
.Submitted script to queue home-scrm. Job ID: 4554890
running 183 tasks as an array-job.
Wrote commands to downsample100.sh
.Submitted script to queue home-scrm. Job ID: 4554891


In [15]:
len(commands)

0

running 500 tasks as an array-job.
job ID: 4356920
running 500 tasks as an array-job.
job ID: 4356921
running 500 tasks as an array-job.
job ID: 4356922
running 500 tasks as an array-job.
job ID: 4356923
running 500 tasks as an array-job.
job ID: 4356924
running 500 tasks as an array-job.
job ID: 4356925
running 400 tasks as an array-job.
job ID: 4356926


In [11]:
commands[:5]

['time seqtk sample -s 1 /home/obotvinnik/projects/singlecell_pnms/data/P9_01_R1.fastq.gz 1000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/P9_01_1000000reads_iteration0_R1.fastq.gz',
 'time seqtk sample -s 1 /home/obotvinnik/projects/singlecell_pnms/data/P9_01_R2.fastq.gz 1000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/P9_01_1000000reads_iteration0_R2.fastq.gz',
 'time seqtk sample -s 2 /home/obotvinnik/projects/singlecell_pnms/data/M6_03_R1.fastq.gz 1000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/M6_03_1000000reads_iteration0_R1.fastq.gz',
 'time seqtk sample -s 2 /home/obotvinnik/projects/singlecell_pnms/data/M6_03_R2.fastq.gz 1000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/M6_03_1000000reads_iteration0_R2.fastq.gz',
 'time seqtk sample -s 3 /home/obotvinnik/projects/singlecell_pnms/data/P9_01_R1.fastq.gz 1000000 | gzip - > /home/obotvinnik/projects/downsample_singlecell/data/P9_01_1000000r

In [12]:
len(commands)

4040

In [13]:
! qstat -q


server: tscc-mgr.local

Queue            Memory CPU Time Walltime Node  Run Que Lm  State
---------------- ------ -------- -------- ----  --- --- --  -----
home-yeo           --      --       --      --   17   7 --   E R
home-eallen        --      --       --      --    0   0 --   E R
gpu-hotel          --      --    336:00:0   --    0   3 --   E R
home-fpaesani      --      --       --      --    0   8 --   E R
home-jogleeson     --      --       --      --    3   8 --   E R
home-tg            --      --       --      --    0   0 --   E R
home-dkeres        --      --       --      --    0   1 --   E R
home-kkey          --      --       --      --    0   0 --   E R
home-builder       --      --       --      --    0   0 --   E R
home-vayyaswamy    --      --       --      --    0   0 --   E R
home-scrm          --      --       --      --    4   0 --   E R
home-mirarab       --      --       --      --    0   0 --   E R
home-ong           --      --       --      --    0   4 --   E 

In [14]:
import qtools

sub = qtools.Submitter(commands, 'downsample', array=True, queue='home-scrm', walltime='01:30:00')

running 500 tasks as an array-job.
job ID: 4346851
running 500 tasks as an array-job.
job ID: 4346852
running 500 tasks as an array-job.
job ID: 4346853
running 500 tasks as an array-job.
job ID: 4346854
running 500 tasks as an array-job.
job ID: 4346855
running 500 tasks as an array-job.
job ID: 4346856
running 500 tasks as an array-job.
job ID: 4346857
running 500 tasks as an array-job.
job ID: 4346858
running 40 tasks as an array-job.
job ID: 4346859


In [1]:
!tail downsample*err*

tail: cannot open `downsample*err*' for reading: No such file or directory


In [4]:
!tail downsample*out*

==> downsample1.sh.out-1 <==
Nodes:        tscc-1-46

==> downsample1.sh.out-10 <==
Nodes:        tscc-1-42

==> downsample1.sh.out-100 <==
Nodes:        tscc-1-35

==> downsample1.sh.out-101 <==
Nodes:        tscc-1-35

==> downsample1.sh.out-102 <==
Nodes:        tscc-1-35

==> downsample1.sh.out-103 <==
Nodes:        tscc-1-35

==> downsample1.sh.out-104 <==
Nodes:        tscc-1-35

==> downsample1.sh.out-105 <==
Nodes:        tscc-1-35

==> downsample1.sh.out-106 <==
Nodes:        tscc-1-35

==> downsample1.sh.out-107 <==
Nodes:        tscc-1-35

==> downsample1.sh.out-11 <==
^C

In [34]:
ls -lha $new_folder

lrwxrwxrwx 1 obotvinnik yeo-group 61 Jan 23 08:01 /home/obotvinnik/projects/downsample_singlecell/data -> /home/obotvinnik/scratch/projects/downsample_singlecell/data//
